# Pregunta 3
Aqui se realizará el desarrollo de la pregunta 3 de Jorge Facuse :D

Primero se creará la clase Node y Leaf, las cuales nos servirán para más adelante crear el Merkle Tree. Notar que son estas las funciones que efectivamente implementan la obtención de la prueba, haciéndolo de manera recursiva.

In [26]:
class Node:
    def __init__(self, direction, parent=None):
        self.direction = direction
        self.parent = parent

    def set_sons(self, sons):
        # Recibe una lista de hijos
        self.sons = sons

    def get_hash(self, hash_fn):    
        return hash_fn(self.sons[0].get_hash(hash_fn) + self.sons[1].get_hash(hash_fn))

    def get_bro(self, hash_fn):
        if self.parent:
            if self.direction == 'i':
                return (self.parent.sons[1].get_hash(hash_fn), 'd')
            else:
                return (self.parent.sons[0].get_hash(hash_fn), 'i')
        return False

    def get_proof(self, hash_fn):
        proof_list = []
        bro = self.get_bro(hash_fn)
        if bro:
            proof_list.append(bro)
            parent_proof = self.parent.get_proof(hash_fn)
            return proof_list + parent_proof
        return proof_list

class Leaf:
    def __init__(self, direction, parent, string):
        self.direction = direction
        self.parent = parent
        self.string = string
    
    def get_hash(self, hash_fn):
        return hash_fn(self.string)

    def get_bro(self, hash_fn):
        if self.direction == 'i':
            return (self.parent.sons[1].get_hash(hash_fn), 'd')
        else:
            return (self.parent.sons[0].get_hash(hash_fn), 'i')

    def get_proof(self, hash_fn):
        proof_list = [self.get_bro(hash_fn)]
        parent_proof = self.parent.get_proof(hash_fn)
        return proof_list + parent_proof


Ahora creamos la clase Merkle Tree. Su funcionamiento es el siguiente: Primero crea el árbol con nodes hasta asegurarse que en el último nivel quepan todos los strings. Luego, rellena el último nivel con los strings, y si se queda sin strings va rellenando con los últimos 2 strings, de manera que siempre se pueda calcular el hash y de el mismo cuando corresponda.

Con esto, implementar get root y get proof es trivial.

In [33]:
class MerkleTree :
    def __init__ ( self , strings, hash_func) :
        """
        Arguments :
        strings : The set of strings S
        """
        self.strings = strings
        self.hash = hash_func
        if len(self.strings) % 2 == 1:
            self.strings.append(self.strings[-1]) # Si es impar, metemos el ultimo de nuevo
        self.root = Node('root')
        i = 1
        next_level_nodes = []
        curr_level_nodes = [self.root]
        while 2**i < len(self.strings):
            # Partimos construyendo el arbol desde la raiz
            for node in curr_level_nodes:
                # Creamos un nodo izquierdo y uno derecho
                right = Node('d', node)
                left = Node('i', node)
                next_level_nodes.append(left)
                next_level_nodes.append(right)
                node.set_sons([left,right])
            curr_level_nodes = next_level_nodes
            next_level_nodes = []
            i += 1
        # Una vez terminamos ese while, quedamos en un nivel en donde el siguiente seran solo hojas
        self.string_to_leaf = {}
        i = 0
        # Comenzamos a rellenar con hojas el arbol
        for node in curr_level_nodes:
            if i < len(self.strings):
                right = Leaf('d', node, self.strings[i+1])
                left = Leaf('i', node, self.strings[i])
                self.string_to_leaf[self.strings[i+1]] = right # Guardamos en que hojas tenemos a nuestros strings
                self.string_to_leaf[self.strings[i]] = left
                node.set_sons([left,right])
                i += 2
            # Siempre van a ser una cantidad de hojas pares, por lo que si nos quedan nodos sin hojas vamos rellenando con las ultimas 2. Asi se cumple que el hash sea el mismo
            else:
                right = Leaf('d', node, self.strings[-1])
                left = Leaf('i', node, self.strings[-2])
                node.set_sons([left,right])

        
    def get_root ( self ) :
        """
        Returns :
        root : Root of the Merkle Tree RETORNA EL HASH
        """
        return self.root.get_hash(self.hash)
    def get_proof_for ( self , item : str ):
        """
        Returns :
        result : None if the item is not part of the leafs of the tree
        A list with the necessary info to prove that the
        item is part of the leafs of the tree
        """
        if not item in self.strings:
            return None
        item_node = self.string_to_leaf[item]
        return item_node.get_proof(self.hash)
        

Definimos una funcion de hash sencilla usando la funcion hash de python para testear

In [7]:
def simple_hash(string):
    return str(hash(string))

In [37]:
tree = MerkleTree(['y', 'que','pasa','le','pregunte', 'por', 'wasa', 'no', 'se', 'quemas', 'viene'], simple_hash)

In [38]:
tree.get_proof_for('le')

[('-6709474068208483061', 'i'),
 ('-3286259426596940793', 'i'),
 ('4586144136951340762', 'd'),
 ('-5700178602063967186', 'd')]

Ahora creamos la función verify. Notemos que esta función asume una proof en el orden correcto, es decir que primero de el hash del hermano de la hoja, luego el del hermano del padre y asi sucesivamente (tal y como se obtiene de get_proof_for).

In [44]:
def verify(
    root,
    item: str,
    proof,
    hash_func
) -> bool:

    """
    Arguments :
    root : The root of a merkle tree
    item : An abritrary string
    proof : An alleged proof that item is part of a Merkle
    tree with root root
    hash_func : An arbitrary hash function
    Returns :
    correct : whether the proof is correct or not
    """
    to_eval = hash_func(item)
    
    # Se asume que 
    for proof_item in proof:
        if proof_item[1] == 'i':
            to_eval = hash_func(proof_item[0] + to_eval)
        else:
            to_eval = hash_func(to_eval + proof_item[0])
    return to_eval == root


In [46]:
root = tree.get_root()
print(verify(root, 'le', [('-6709474068208483061', 'i'),
                          ('-3286259426596940793', 'i'),
                          ('4586144136951340762', 'd'),
                          ('-5700178602063967186', 'd')], simple_hash)) # Esto deberia dar correcto
print(verify(root, 'le', [('-6709474068208483061', 'i'),
                          ('-3286229426596940793', 'i'),
                          ('4586144136951340762', 'd'),
                          ('-5700178602063967186', 'd')], simple_hash)) # Esto deberia dar falso

[('-6709474068208483061', 'i'), ('-3286259426596940793', 'i'), ('4586144136951340762', 'd'), ('-5700178602063967186', 'd')] 2321299573829070902
[('-6709474068208483061', 'i'), ('-3286259426596940793', 'i'), ('4586144136951340762', 'd'), ('-5700178602063967186', 'd')] -2312340029754931672
True
[('-6709474068208483061', 'i'), ('-3286229426596940793', 'i'), ('4586144136951340762', 'd'), ('-5700178602063967186', 'd')] -9016507504311716024
[('-6709474068208483061', 'i'), ('-3286229426596940793', 'i'), ('4586144136951340762', 'd'), ('-5700178602063967186', 'd')] -881445457956480946
False
